# $\text{HeteroGATomics Google Colab Demo}$

| [Preprint](https://arxiv.org/abs/2408.02845) | [Github Repo](https://github.com/SinaTabakhi/HeteroGATomics) | [Run in Colab](https://colab.research.google.com/github/SinaTabakhi/HeteroGATomics/blob/main/HeteroGATomics_demo.ipynb) (click `Runtime` → `Run all (Ctrl+F9)` |

This notebook presents **HeteroGATomics**, a novel framework employing heterogeneous graph attention networks for integrating multiomics data for cancer diagnosis.

In [ ]:
# @title ## Setting up the notebook environment (~4min)

#@markdown Executing this code block is necessary for setting up the notebook environment. It automatically installs all required packages when the notebook is used on Google Colab.
import os
import torch

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

if 'google.colab' in str(get_ipython()):
    print('Running on Google Colab...')
    !pip install pandas==2.1.1
    !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
    !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
    !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
    !pip install lightning
    !pip install yacs
    !git clone https://github.com/SinaTabakhi/HeteroGATomics.git
    %cd HeteroGATomics
else:
    print('Not running on Google Colab...')

In [2]:
# @title ## Importing necessary functions
import os
import time
import argparse
import warnings
import pickle
import gzip
import copy
import logging
from collections import defaultdict

import numpy as np
import pytorch_lightning as pl
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from concurrent.futures import ProcessPoolExecutor as Pool
from xgboost import XGBClassifier

from raw_data_loader import MultiOmicsDataset
from gat.graph_data_loader import HeteroDataset
from gat.architecture import NewModel
from feature_selection.multi_agent_system import MultiAgentSystem
from utils import seed_everything, evaluate_model, select_top_feats, load_dataset_indices, create_file
from configs import get_cfg_defaults

In [ ]:
# @title ## Customizing the configuration of HeteroGATomics
#@markdown The customized configuration used in this notebook is stored in [configs/HeteroGATomics_Demo.yaml](https://github.com/SinaTabakhi/HeteroGATomics/blob/main/configs/HeteroGATomics_Demo.yaml). This file overwrites default parameter values specified in [config.py](https://github.com/SinaTabakhi/HeteroGATomics/blob/main/configs.py).

#@markdown By default, HeteroGATomics will be trained on the Brain Lower Grade Glioma (LGG) dataset. Alternatively, you can choose from two other datasets for your experiments.
dataset_name = "Brain Lower Grade Glioma (LGG)" #@param ["Bladder Urothelial Carcinoma (BLCA)", "Brain Lower Grade Glioma (LGG)", "Renal Cell Carcinoma (RCC)"]
#@markdown By default, HeteroGATomics runs on just one fold due to the resource constraints of a free Colab account. You can increase this value to 10 if you have sufficient resources.
num_folds = 1 # @param {type:"integer"}

# DNA_feature_sparsity_rate = 0.85 # @param {type:"number"}
# mRNA_feature_sparsity_rate = 0.85 # @param {type:"number"}
# miRNA_feature_sparsity_rate = 0.85 # @param {type:"number"}
# save_outputs = False #@param {type:"boolean"}
# save_trained_models = False #@param {type:"boolean"}

num_folds = num_folds if num_folds <= 10 else 10

if dataset_name == "Bladder Urothelial Carcinoma (BLCA)":
  dataset_name = "BLCA"
  num_classes = 2
elif dataset_name == "Brain Lower Grade Glioma (LGG)":
  dataset_name = "LGG"
  num_classes = 2
elif dataset_name == "Renal Cell Carcinoma (RCC)":
  dataset_name = "RCC"
  num_classes = 3

warnings.filterwarnings(action="ignore")
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)
logging.getLogger("lightning").setLevel(logging.ERROR)

cfg_path = "./configs/HeteroGATomics_Demo.yaml"
fold_max_workers = 1
agent_max_workers = 1

cfg = get_cfg_defaults()
cfg.merge_from_file(cfg_path)
cfg.freeze()
seed_everything(cfg.SOLVER.SEED, workers=True)

# ---- setup folders and paths ----
if not os.path.exists(cfg.RESULT.SAVE_RICH_DATA_DIR) and cfg.RESULT.SAVE_RICH_DATA:
    os.makedirs(cfg.RESULT.SAVE_RICH_DATA_DIR)
if not os.path.exists(cfg.RESULT.SAVE_MODEL_DIR) and cfg.RESULT.SAVE_MODEL:
    os.makedirs(cfg.RESULT.SAVE_MODEL_DIR)

main_folder = os.path.join(cfg.DATASET.ROOT, dataset_name)
raw_file_paths = [(os.path.join(main_folder, f"{omics}.csv"), omics) for omics in cfg.DATASET.OMICS]
raw_label_path = os.path.join(main_folder, f"ClinicalMatrix.csv")

print(f"Config yaml: {cfg_path}")

In [4]:
# @title ## Loading dataset
#@markdown The training and test sets for each fold of the 10-fold cross-validation are loaded, and a multiomics dataset is created using the `MultiOmicsDataset` class.
def prepare_data(main_folder, fold_idx, multiomics, cfg, dataset_name, feat_selection=True):
    fold_dir = os.path.join(main_folder, f"{fold_idx + 1}")
    train_index, test_index = load_dataset_indices(fold_dir)
    multiomics_copy = copy.deepcopy(multiomics)

    train_index.sort()
    test_index.sort()
    multiomics_copy.set_train_test(train_index, test_index)
    multiomics_copy.config_components()

    if not feat_selection:
      load_data_name = cfg.RESULT.SAVE_RICH_DATA_TMPL.format(dataset_name=dataset_name, fold_idx=fold_idx + 1)

      with gzip.open(os.path.join(cfg.RESULT.SAVE_RICH_DATA_DIR, load_data_name), 'rb') as file:
          loaded_data = pickle.load(file)
          loaded_node_pheromones, loaded_edge_pheromones = loaded_data
          multiomics_copy.set_data_structure(loaded_node_pheromones, loaded_edge_pheromones)

    return multiomics_copy


# ---- setup multiomics dataset ----
multiomics = MultiOmicsDataset(
    dataset_name=dataset_name,
    raw_file_paths=raw_file_paths,
    raw_label_path=raw_label_path,
    num_omics=len(cfg.DATASET.OMICS),
    num_classes=num_classes,
    init_pheromone_val=cfg.ACO.INIT_PHEROMONE,
    sparsity_rates=cfg.DATASET.FEATURE_SPARSITY_RATES
)
print(multiomics)

fold_multiomics = []
for fold_idx in range(num_folds):
    print(f"==> Loading data from fold {fold_idx + 1}...")
    fold_multiomics.append(prepare_data(main_folder, fold_idx, multiomics, cfg, dataset_name))


Dataset info:
   dataset name: LGG
   number of omics: 3
   number of classes: 2

   omics    | num samples | num features
   ----------------------------------------
   DNA      | 522         | 8277        
   mRNA     | 522         | 1166        
   miRNA    | 522         | 287         
   ----------------------------------------


==> Loading data from fold 1...


# Performing feature selection

Here, we implement joint feature selection using a multi-agent system to identify a subset of informative features. The hyperparameters of the algorithm are defined as follows:

- **max_iterations**: Maximum number of iterations.
- **num_agents**: The number of agents used in each modality.
- **fix_feature_size**: The number of selected features in each iteration by each agent.
- **node_desirability_decay_rate**: Node desirability decay rate.
- **edge_desirability_decay_rate**: Edge desirability decay rate.
- **modality_importance_decay_rate**: Modality importance decay rate.
- **control_parameter_𝑞0**: The constant parameter in the state transition rules.

In [5]:
# @title ## MAS hyperparameters
max_iterations = 1 # @param {type:"integer"}
num_agents = 1 # @param {type:"integer"}
fix_feature_size = 2 # @param {type:"integer"}
node_desirability_decay_rate = 0.1 # @param {type:"number"}
edge_desirability_decay_rate = 0.1 # @param {type:"number"}
modality_importance_decay_rate = 0.1 # @param {type:"number"}
control_parameter_𝑞0 = 0.8 # @param {type:"number"}

def select_features(args):
    fold_idx, multiomics, agent_max_workers, cfg = args

    start_time = time.time()
    multi_agent = MultiAgentSystem(
        max_workers=agent_max_workers,
        dataset=multiomics,
        max_iters=max_iterations,
        num_agents=num_agents,
        num_feats=fix_feature_size,
        q0=control_parameter_𝑞0,
        node_discount_rate=node_desirability_decay_rate,
        edge_discount_rate=edge_desirability_decay_rate,
        prob_discount_rate=modality_importance_decay_rate
    )

    print(f"\n==> Performing feature selection for fold {fold_idx + 1}...")
    print(multi_agent)
    multi_agent.run()

    end_time = time.time()
    running_time = end_time - start_time

    return fold_idx, running_time, multiomics


with Pool(max_workers=fold_max_workers) as pool:
    fold_results = pool.map(select_features, [(fold_idx, fold_multiomics[fold_idx], agent_max_workers, cfg)
                                              for fold_idx in range(num_folds)])

cls_results = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
time_results = defaultdict(list)

print(f"\n==> Collecting information...")
for fold_idx, running_time, rich_multiomics in fold_results:
    node_pheromones = rich_multiomics.get_node_pheromone()
    node_relevances = rich_multiomics.get_node_relevance()
    edge_pheromones = rich_multiomics.get_edge_pheromone()

    if cfg.RESULT.SAVE_RICH_DATA:
        save_object = [node_pheromones, edge_pheromones]
        save_data_name = cfg.RESULT.SAVE_RICH_DATA_TMPL.format(dataset_name=dataset_name, fold_idx=fold_idx + 1)
        with gzip.open(os.path.join(cfg.RESULT.SAVE_RICH_DATA_DIR, save_data_name), 'wb') as file:
            pickle.dump(save_object, file)

    for feat_size in cfg.ACO.FINAL_FEAT_SIZES:
        print(f"  ==> Fold {fold_idx + 1} - feature size {feat_size}...")
        rich_multiomics_deepcopy = copy.deepcopy(rich_multiomics)
        start_time = time.time()
        final_feat_subset = select_top_feats(node_pheromones, node_relevances, feat_size, multiomics.num_omics,
                                              cfg.ACO.SELECTION_RATE)
        end_time = time.time()
        running_time += end_time - start_time

        rich_multiomics_deepcopy.reduce_dimensionality(final_feat_subset)
        final_train_data = rich_multiomics_deepcopy.concatenate_data(is_train=True)
        final_test_data = rich_multiomics_deepcopy.concatenate_data(is_train=False)

        models = [
            RandomForestClassifier(),
            XGBClassifier(),
            KNeighborsClassifier(),
            MLPClassifier(max_iter=500),
            RidgeClassifier()
        ]

        for model in models:
            model_result = evaluate_model(
                model,
                train_data=final_train_data.values,
                test_data=final_test_data.values,
                train_label=rich_multiomics_deepcopy.get(0).train_label.values.ravel(),
                test_label=rich_multiomics_deepcopy.get(0).test_label.values.ravel(),
                num_classes=rich_multiomics_deepcopy.num_classes
            )

            for metric_name, metric_value in model_result.items():
                cls_results[feat_size][model.__class__.__name__][metric_name].append(metric_value)

        time_results[feat_size].append(running_time)


print(f"\n==> Showing results...")
for feat_size, models in cls_results.items():
    exe_time = round(np.mean(time_results[feat_size]) / min(num_folds, fold_max_workers))
    print(f"Feature size {feat_size} (execution time: {exe_time} seconds)")
    for model_name, metrics in models.items():
        print(f"    {model_name}:")
        for metric_name, values in metrics.items():
            average = np.mean(values)
            std = np.std(values)
            print(f"        - {metric_name}: {average:.3f}±{std:.3f}")


==> Performing feature selection for fold 1...

    Multi-agent system configurations
        Number of iterations:           1
        Number of selected features:    2     
        q0:                             0.8
        Node discount rate:             0.1
        Edge discount rate:             0.1
        Omics importance discount rate: 0.1
        Number of agents per omics:     1
        Total number of agents:         3

    ------------------------------- Iteration 1 -------------------------------
                 ---------- Current selected feature 1 --------------------- 
                 ---------- Current selected feature 2 --------------------- 

==> Collecting information...
  ==> Fold 1 - feature size 300...

==> Showing results...
Feature size 300 (execution time: 86 seconds)
    RandomForestClassifier:
        - AUROC: 0.755±0.000
        - Accuracy: 0.755±0.000
        - NPV: 0.741±0.000
        - PPV: 0.769±0.000
        - Sensitivity: 0.741±0.000
        - Spe

# Executing the graph attention network (GAT) module

Here, we perform label prediction by employing graph attention networks over constructed heterogeneous graphs. The hyperparameters of the algorithm are defined as follows:

- **max_epochs_pretrain**: The number of epochs for pre-training model.
- **max_epochs**: The number of epochs for training model.
- **num_heads**: The number of multi-head attentions.
- **dropout_rate**: Dropout probability.
- **learning_rate_pretrain**: The learning rate used in the GAT module for the pre-training model.
- **learning_rate**: The learning rate used in the GAT module for the training model.
- **VCDN_learning_rate**: The learning rate used in the VCDN module.
- **hidden_dimensions**: Hidden node dimensions for each layer.


In [6]:
# @title ## GAT hyperparameters
max_epochs_pretrain = 5 # @param {type:"integer"}
max_epochs = 5 # @param {type:"integer"}
num_heads = 2 # @param {type:"integer"}
dropout_rate = 0.1 # @param {type:"number"}
learning_rate_pretrain = 0.001 # @param {type:"number"}
learning_rate = 0.001 # @param {type:"number"}
VCDN_learning_rate = 0.05 # @param {type:"number"}
hidden_dimensions = [100, 100, 50] # @param {type:"raw"}
num_layers = len(hidden_dimensions)

gat_results = defaultdict(lambda: defaultdict(list))
time_results = defaultdict(list)

for fold_idx in range(num_folds):
    print(f"==> Loading data from fold {fold_idx + 1}...")
    fold_multiomics = prepare_data(main_folder, fold_idx, multiomics, cfg, dataset_name, feat_selection=False)
    node_pheromones = fold_multiomics.get_node_pheromone()
    node_relevances = fold_multiomics.get_node_relevance()

    for feat_size in cfg.GAT.FINAL_FEAT_SIZES:
        final_feat_subset = select_top_feats(node_pheromones, node_relevances, feat_size, fold_multiomics.num_omics,
                                              cfg.ACO.SELECTION_RATE)

        multiomics_deepcopy = copy.deepcopy(fold_multiomics)
        multiomics_deepcopy.reduce_dimensionality(final_feat_subset, feat_size)

        start_time = time.time()
        hetero_data = HeteroDataset(multiomics_deepcopy, cfg.DATASET.PATIENT_SPARSITY_RATES,
                                    tune_hyperparameters=cfg.SOLVER.TUNE_HYPER,
                                    seed=cfg.SOLVER.SEED)
        hetero_data.create_hetero_data()

        # ---- setup model ----
        print("\n   ==> Building model...")
        new_model = NewModel(dataset=hetero_data,
                              num_modalities=multiomics_deepcopy.num_omics,
                              num_classes=multiomics_deepcopy.num_classes,
                              gat_num_layers=num_layers,
                              gat_num_heads=num_heads,
                              gat_hidden_dim=hidden_dimensions,
                              gat_dropout_rate=dropout_rate,
                              gat_lr_pretrain=learning_rate_pretrain,
                              gat_lr=learning_rate,
                              gat_wd=cfg.GAT.WD,
                              vcdn_lr=VCDN_learning_rate,
                              vcdn_wd=cfg.VCDN.WD,
                              tune_hyperparameters=cfg.SOLVER.TUNE_HYPER
                              )

        # ---- setup pretraining model and trainer ----
        print("\n   ==> Pretrain model...")
        model = new_model.get_model(pretrain=True)
        trainer_pretrain = pl.Trainer(
            max_epochs=max_epochs_pretrain,
            default_root_dir=cfg.RESULT.LIGHTNING_LOG_DIR,
            accelerator="auto",
            devices="auto",
            enable_model_summary=False,
            log_every_n_steps=1
        )
        trainer_pretrain.fit(model)

        # ---- setup training model and trainer ----
        print("\n   ==> Training model...")
        model = new_model.get_model(pretrain=False)
        trainer = pl.Trainer(
            max_epochs=max_epochs,
            default_root_dir=cfg.RESULT.LIGHTNING_LOG_DIR,
            accelerator="auto",
            devices="auto",
            enable_model_summary=False,
            log_every_n_steps=1
        )
        trainer.fit(model)

        # ---- test model ----
        print("\n   ==> Testing model...")
        trainer.test(model)
        end_time = time.time()
        running_time = end_time - start_time

        time_results[feat_size].append(running_time)

        for metric_key, metric_value in model.get_log_metrics().items():
            if metric_key.startswith("test_"):
                gat_results[feat_size][metric_key.replace("test_", "")].append(metric_value[0])


print(f"\n==> Showing results...")
for feat_size, metrics in gat_results.items():
    exe_time = round(np.mean(time_results[feat_size]))
    print(f"Feature size {feat_size} (execution time: {exe_time} seconds)")
    for metric_name, values in metrics.items():
        average = np.mean(values)
        std = np.std(values)
        print(f"    - {metric_name}: {average:.3f}±{std:.3f}")

==> Loading data from fold 1...

   ==> Building model...

   ==> Pretrain model...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


   ==> Testing model...


Testing: |          | 0/? [00:00<?, ?it/s]


==> Showing results...
Feature size 300 (execution time: 53 seconds)
    - AUROC: 0.285±0.000
    - Accuracy: 0.491±0.000
    - NPV: 0.491±0.000
    - PPV: 0.000±0.000
    - Sensitivity: 0.000±0.000
    - Specificity: 1.000±0.000
